In [1]:
from benchmark import *
%matplotlib inline

In [2]:
helpers = 'pdb'
mcss = '../../../mcss_sizes.pkl'

sp = 'stats26'
xp = 'stats20'

def results(mcss_func, name, aggregate):
    data = pd.concat(load(version, helpers, mcss) for version in [sp, xp])
    data = add_correct(data, thresh = 2.0)
    data = data[(data.index.get_level_values('protein') != 'A2AR')]
    data = data[mcss_func(data.mcss)]

    ligand = data.xs((helpers, 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                      level=('helpers', 'mode', 'n_ligs', 'features', 'alpha')).filter(regex='rmsd|correct')
    ligand = filter_to_ubiquitous_ligands(ligand)
    target = ligand.groupby(level=list(range(len(ligand.index.levels)-1))).mean().dropna()
    
    if aggregate == 'target':
        family = target.groupby(level=list(range(len(target.index.levels)-1))).mean()
    elif aggregate == 'ligand':
        family = ligand.groupby(level=list(range(len(ligand.index.levels)-2))).mean()

    target = pd.merge(target.loc[sp], target.loc[xp], left_index=True, right_index=True)
    family = pd.merge(family.loc[sp], family.loc[xp], left_index=True, right_index=True)
    
    
    

    family['SP Top'] = family['glide_correct_x']
    family['SP ComBind'] = family['combind_correct_x']
    family['SP Best'] = family['best_correct_x']
    family['XP Top'] = family['glide_correct_y']
    family['XP ComBind'] = family['combind_correct_y']
    family['XP Best'] = family['best_correct_y']
    family = family.filter(regex='P')
    
    family['version'] = name
    family.reset_index(inplace=True)
    family.set_index(['version', 'family'], inplace = True)
    return drug_average(family)

In [3]:
pd.concat([results(lambda x: x > -1, 'All', 'target'),
           results(lambda x: x >= 0.5, 'Shared scaffold', 'target'),
           results(lambda x: x < 0.5, 'No shared scaffold', 'target')])

,SP Top,SP ComBind,SP Best,XP Top,XP ComBind,XP Best
version,,,,,,
All,0.462818,0.603577,0.798122,0.499603,0.538024,0.644767
Shared scaffold,0.498975,0.623802,0.873302,0.591418,0.741800,0.792294
No shared scaffold,0.412037,0.544746,0.790784,0.431991,0.476311,0.585132


In [4]:
pd.concat([results(lambda x: x > -1, 'All', 'ligand'),
           results(lambda x: x >= 0.5, 'Shared scaffold', 'ligand'),
           results(lambda x: x < 0.5, 'No shared scaffold', 'ligand')])

,SP Top,SP ComBind,SP Best,XP Top,XP ComBind,XP Best
version,,,,,,
All,0.461617,0.604941,0.808457,0.512051,0.563768,0.679999
Shared scaffold,0.576191,0.757223,0.859450,0.708593,0.774531,0.823999
No shared scaffold,0.409612,0.536365,0.768007,0.433379,0.476933,0.615341


# Number of ligands for each target

In [5]:
data = pd.concat(load(version, helpers, mcss) for version in [sp, xp])
data = add_correct(data, thresh = 2.0)

all_ligands = data.xs(('stats21', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                  level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

diverse = data[data.mcss < 0.5].xs(('stats21', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                  level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

diverse_correct = data[(data.mcss < 0.5) & data.best_correct].xs(
                    ('stats21', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                     level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

df = pd.DataFrame()
df['all'] = all_ligands.groupby(level='protein').count().iloc[:, 0]
df['diverse'] = diverse.groupby(level='protein').count().iloc[:, 0]
df['diverse_correct'] = diverse_correct.groupby(level='protein').count().iloc[:, 0]
df

KeyError: 'stats21'

In [ ]:
df.sum()